In [1]:
import traceback
import logging
import typing as t

import os
import shutil
import matplotlib.pyplot as plt

from pathlib import Path

import pandas as pd
import numpy as np
import altair as alt
import seaborn as sns

from importlib import reload

from IPython.display import clear_output

# from jupyterthemes import jtplot
# jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

logger = logging.getLogger()
logger.setLevel(logging.INFO)

%matplotlib inline
%config InlineBackend.figure_format='retina'
%load_ext autoreload
%autoreload 2
alt.data_transformers.disable_max_rows()
sns.set_theme()

In [1097]:
from functools import partial
import itertools
from pathlib import Path

import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import neptune.new as neptune

from battery_optimizations.date_splitting import get_day_load, get_interval_load
from battery_optimizations.reference_signal import calculate_ref_load

from mpc_optimization.battery_model_constants import Battery_state_vars, State_vars_aliases_dict, C_rate_per_second
from mpc_optimization.fmu_source import FmuSource, ModelicaModelInfo
from mpc_optimization.mpc_optimizer import MPCOptimizer, ModelVariables, VariableConstraint

mpc_optimizer = MPCOptimizer(
    model_info=ModelicaModelInfo(Path("/home/developer/modelica/BatteryWithFullCycle.mo"),
                                 "BatteryMPC.BatteryWithFullCycle"),
#     fmu_path="/home/developer/ipynotebooks/BatteryMPC.BatteryWithFullCycle.fmu",
    initial_parameters={
        'theveninBasedBattery.coulombSocCounter.SOC_init.k': 0.02
    },
    state_variables=Battery_state_vars,
    input_vec=['I_req'],
    output_vec=['SoC', 'SoH'],
    points_per_sec=.1
)

Notification: Automatically loaded package Modelica 3.2.3 due to uses annotation.
Notification: Automatically loaded package Complex 3.2.3 due to uses annotation.
Notification: Automatically loaded package ModelicaServices 3.2.3 due to uses annotation.

LOG_SUCCESS       | info    | The initialization finished successfully without homotopy method.
LOG_SUCCESS       | info    | The simulation finished successfully.


In [1243]:
start = 360
end = 60*60

control_df = pd.DataFrame({"Time": range(start, end)})
# control_df['I_req'] = -3e-4/4
control_df['I_req'] = 0
# control_df['I_req'] = control_df['Time'].apply(lambda x: -1*x/360*1e-6)
control_df.set_index('Time', inplace=True)

sim_res = mpc_optimizer.simulate(start, end, control_df)

In [1107]:
list(filter(lambda x: x.startswith(''), sim_res[0].columns));

In [1244]:
output = sim_res[0][['theveninBasedBattery.R_ts.R', 'theveninBasedBattery.R_tl.R', 'theveninBasedBattery.C_ts.C', 'theveninBasedBattery.C_tl.C', 'theveninBasedBattery.R_tl.v', 'theveninBasedBattery.R_ts.v', 'SoC', 'theveninBasedBattery.U_oc.v']]

list(output.columns) #['']
output

,theveninBasedBattery.R_ts.R,theveninBasedBattery.R_tl.R,theveninBasedBattery.C_ts.C,theveninBasedBattery.C_tl.C,theveninBasedBattery.R_tl.v,theveninBasedBattery.R_ts.v,SoC,theveninBasedBattery.U_oc.v
0,96.489879,880.129489,0.115961,0.508011,0.0,0.0,0.02,2.86745
1,96.489879,880.129489,0.115961,0.508011,0.0,0.0,0.02,2.86745
2,96.489879,880.129489,0.115961,0.508011,0.0,0.0,0.02,2.86745
3,96.489879,880.129489,0.115961,0.508011,0.0,0.0,0.02,2.86745
4,96.489879,880.129489,0.115961,0.508011,0.0,0.0,0.02,2.86745
...,...,...,...,...,...,...,...,...
320,96.489879,880.129489,0.115961,0.508011,0.0,0.0,0.02,2.86745
321,96.489879,880.129489,0.115961,0.508011,0.0,0.0,0.02,2.86745
322,96.489879,880.129489,0.115961,0.508011,0.0,0.0,0.02,2.86745
323,96.489879,880.129489,0.115961,0.508011,0.0,0.0,0.02,2.86745


In [1245]:
i = sim_res[1]['I_req'].values
v = sim_res[2]['batteryOutput'].values
u_ocv = sim_res[0]['theveninBasedBattery.U_oc.v'].values
socs_true = sim_res[2]['SoC']
T = np.diff(sim_res[0]['time'].values, 1)[0]

In [1205]:
def V_ocv(soc):
    a = [-5.863e-1, 21.9, 3.414, 1.102e-1, -1.718e-1, 8.0e-3]
#     return 3.685+0.2156*soc - 0.1178*soc**2 + 0.3201*soc**3 - 1.031 * np.e**(-35*soc)
    return a[0]*np.exp(-a[1]*soc) + a[2] + a[3]*soc + a[4]*np.exp(-a[5]/(1-soc)) 


# Estimation using RLS

In [471]:
import padasip as pa

v_b = 2.65727

X = np.concatenate([(pa.input_from_history(v, 3)), 
                     pa.input_from_history(i, 3),
                     pa.input_from_history(output.SoC.values, 3),
                   ], axis=1)

rls = pa.filters.FilterRLS(n=5, mu=0.93, w='zeros')

res = []

for x in X:
    
    inp = x[1:-3]
    inp[0] -= V_ocv(x[-2])
    inp[1] -= V_ocv(x[-1])
    
    y = rls.predict(inp)
    y_true = V_ocv(x[-3]) - x[0]
    rls.adapt(y_true, inp)
    
    k = np.concatenate([np.ones(1), rls.w])
    k[0] = T*T/(k[1]+k[2]+1)
    
    a = k[0]*k[2]
    b = -k[0]*(k[1]+2*k[2])/T
    c = k[0]*(k[3]+k[4]+k[5])/(T*T)
    d = -k[0]*(k[4]+2*k[5])/T
    
    R_i = k[5]/k[2]
    t_1 = (b + (b*b - 4*a)**(0.5))/2
    t_2 = (b - (b*b - 4*a)**(0.5))/2
    
    R_1 = (t_1*c + t_2*R_i - d)/(t_1 - t_2)
    R_2 = c - R_1 - R_i
    C_1 = t_1/R_1
    C_2 = t_2/R_2
    res.append((y, y_true, R_1, R_2, C_1, C_2)) 
    

## Visualize RLS results

In [472]:
ys, ys_true, Rs_1, Rs_2, Cs_1, Cs_2 = list(zip(*res))
time = sim_res[0]['time'].values[2:]

In [473]:
df = pd.DataFrame({'y': ys, 'time': time})
df['type'] = 'y'

df_true = pd.DataFrame({'y': ys_true, 'time': time})
df_true['type'] = 'y_true'

df = pd.concat([df, df_true])

alt.Chart(df).mark_line().encode(
    x='time:Q',
    y='y:Q',
    color='type:N'
)

alt.Chart(...)

In [474]:
df = pd.concat([
    pd.DataFrame({'y': Rs_1, 'time': time, 'type': 'R_1'}),
    pd.DataFrame({'y': Rs_2, 'time': time, 'type': 'R_2'}),
])

alt.Chart(df).mark_line().encode(
    x='time:Q',
    y='y:Q',
    color='type:N'
)

alt.Chart(...)

In [475]:
df = pd.concat([
    pd.DataFrame({'y': Cs_1, 'time': time, 'type': 'C_1'}),
    pd.DataFrame({'y': Cs_2, 'time': time, 'type': 'C_2'})
])

alt.Chart(df).mark_line().encode(
    x='time:Q',
    y='y:Q',
    color='type:N'
)

alt.Chart(...)

In [476]:
R_1, R_2, C_1, C_2

(32.783730782567915,
 0.005942572671060933,
 550.4965510905133,
 -567.5650470467209)

# Estimation using Kalman Filter

In [1246]:
from filterpy.kalman import ExtendedKalmanFilter as Ekf
from filterpy.kalman import KalmanFilter, predict, update

param_est_filter = KalmanFilter(dim_x=4, dim_z=1)
param_est_filter.Q = np.diag([0.1, 1e-7, 1e-7, 1e-13])
param_est_filter.R = np.ones((1, 1))*3e-2
param_est_filter.P = np.identity(4)*1e5

param_est_filter.x = np.array([0.99, 6.1e-3, -1.2e-2, 6.1e-3])

param_est_filter.F = np.identity(4)
param_est_filter.B = np.zeros(4)


ocv_est_filter = KalmanFilter(dim_x=2, dim_z=1)
ocv_est_filter.Q = np.diag([1e-6, 1e-4])
ocv_est_filter.R = np.ones((1, 1))*3e-7
ocv_est_filter.P = np.diag([1, 100])

ocv_est_filter.x = np.array([3.5, 0])

ocv_est_filter.H = np.ones((1, 2))

inp = np.vstack([
    np.diff(v, 1, prepend=0),
    np.roll(i, 2),
    np.roll(i, 1),
    np.roll(i, 0),
]).T[2:]

res = []

for v_k, phi in zip(v[2:], inp):
    param_est_filter.predict()
    param_est_filter.update(v_k, H=phi.reshape(-1, 4))
    
    a2, b0, b1, b2 = param_est_filter.x
    
    R_0 = b0
    R_1 = (a2*a2*b0+a2*b1+b2)/(1-a2)**2
    C_1 = T*(1-a2)/(a2*a2*b0+a2*b1+b2)
    C_ocv = T*(1-a2)/(b0+b1+b2)
    
    y = phi.reshape(-1, 4).dot(param_est_filter.x)
    
    ocv_est_filter.F = np.diag([1, 1 - T/(R_1*C_1)])
    ocv_est_filter.B = np.array([T/C_ocv, T/C_1])
    
    ocv_est_filter.predict(u=phi[1])
    ocv_est_filter.update(v_k-R_0*phi[1])
    v_ocv = ocv_est_filter.x[0]
#     print(ocv_est_filter.x)
    
    
    res.append((y, v_k, R_0, R_1, C_1, C_ocv, v_ocv))

In [1247]:
ys, ys_true, Rs_1, Rs_2, Cs_1, Cs_2, vs_ocv = list(zip(*res))
time = sim_res[0]['time'].values[2:]

In [1248]:
lookup_table = []
for i in np.linspace(1e-6, 1-1e-6, 1000):
    lookup_table.append((i, V_ocv(i)))
# vs_ocv
lookup_table = np.array(lookup_table)

socs = []

for v in vs_ocv:
    soc = lookup_table[np.argmin( np.abs(lookup_table[:, 1] - v) ), 0]
    socs.append(soc)

In [1249]:
df = pd.concat([
    pd.DataFrame({'y': socs, 'time': time, 'type': 'SoC'}),
    pd.DataFrame({'y': socs_true[2:], 'time': time, 'type': 'SoC_true'}),
])

alt.Chart(df).mark_line().encode(
    x='time:Q',
    y=alt.Y('y:Q', scale=alt.Scale(domain=[0, 1], clamp=True)),
    color='type:N'
)

alt.Chart(...)

In [1250]:
df = pd.concat([
    pd.DataFrame({'y': vs_ocv, 'time': time, 'type': 'U_ocv'}),
    pd.DataFrame({'y': u_ocv[2:], 'time': time, 'type': 'U_ocv_true'}),
])

alt.Chart(df).mark_line().encode(
    x='time:Q',
    y=alt.Y('y:Q', scale=alt.Scale(domain=[1, 4.5], clamp=True)),
    color='type:N'
)

alt.Chart(...)

In [1251]:
df = pd.concat([
    pd.DataFrame({'y': Rs_1, 'time': time, 'type': 'R_0'}),
    pd.DataFrame({'y': Rs_2, 'time': time, 'type': 'R_1'}),
])

alt.Chart(df).mark_line().encode(
    x='time:Q',
    y='y:Q',
    color='type:N'
)

alt.Chart(...)

In [1252]:
df = pd.concat([
    pd.DataFrame({'y': Cs_1, 'time': time, 'type': 'C_1'}),
    pd.DataFrame({'y': Cs_2, 'time': time, 'type': 'C_ocv'})
])

alt.Chart(df).mark_line().encode(
    x='time:Q',
    y='y:Q',
    color='type:N'
)

alt.Chart(...)

In [1253]:
df = pd.DataFrame({'y': ys, 'time': time})
df['type'] = 'y'

df_true = pd.DataFrame({'y': ys_true, 'time': time})
df_true['type'] = 'y_true'

df = pd.concat([df, df_true])

alt.Chart(df).mark_line().encode(
    x='time:Q',
    y=alt.Y('y:Q', scale=alt.Scale(domain=[2.5, 7])),
    color='type:N'
)

alt.Chart(...)